# List BQ objects at the organization level

Luis Gerardo Baeza, August 2024

[The BigQuery Documentation](https://cloud.google.com/bigquery/docs/information-schema-views#required_permissions) explains how to retrieve objects (such as views and tables at the project level or dataset level). However, for large organizations handling several projects, sometimes this is needed at the organizational level. This scripts creates a pandas dataframe with the objects (tables or views) by iterating all projects.

### Permissions required
* Browser permission on the organization
* BigQuery Metadata Viewer (roles/bigquery.metadataViewer) on each project

In [ ]:
pip install google-cloud-resource-manager

In [76]:
from google.cloud import bigquery
from google.cloud import resourcemanager
import pandas as pd

In [107]:
rm_client = resourcemanager.ProjectsClient()
bq_client = bigquery.Client()
regions = ["us-east1", "us"] # Replace with your own region IDs
org_id = "" # Insert with your organization ID
object_type = "TABLES" # VIEWS or TABLES

In [103]:
projects_list = []
projects = list(rm_client.list_projects(parent=f"organizations/{org_id}"))
for project in projects:
    projects_list.append(project.project_id)

In [ ]:
all_bq_objects = []
    
for region in regions:
    print(region)

    for project_id in projects_list:
        print(project_id)

        query = f"""
            SELECT 
              table_catalog as project_id,
              table_schema as dataset,
              table_name as view_name
            FROM `{project_id}.region-{region}.INFORMATION_SCHEMA.{object_type}`
        """
        print(query)

        query_job = bq_client.query(query)
        results = [dict(row) for row in query_job.result()]
        all_bq_objects += results

In [110]:
f"{len(all_bq_objects)} objects found across {len(projects)} projects in {len(region)} regions"

'91 objects found across 7 projects in 2 regions'

In [ ]:
bq_objects = pd.DataFrame(all_bq_objects)
bq_objects